In [15]:
%load_ext autoreload
%autoreload 2
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import Vantage
from pylabrobot.resources.hamilton import VantageDeck
backend = VantageDeck(size=1.3)
lh = LiquidHandler(backend=Vantage(), deck=backend)
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
q_bit = pd.read_csv('/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test.csv')
print(q_bit)

  column/row    1    2    3    4    5    6    7    8    9   10   11   12
0          A  170   57   87  102  175   38   56  178  189   32   95   50
1          B   77  128   78  174   58   88  192  178  109  157  131   27
2          C   84  192  127   49   55  157  120   84   50  179  193  168
3          D  125  162  149  110   54  115  105   92  100  157  113  108
4          E   61   36  160  111  162   36   45   96   26  165   76  171
5          F  135  102   95   98   41  103  106   45  102  178   96  126
6          G   64  174  152  137   79   29  179  199  112  199  115  115
7          H  100  111   28   99   25   31  150   48  161   59   21   74


In [20]:

input_file_path = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test.csv'
diluted = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test_transformed.csv'

df = pd.read_csv(input_file_path)


def transform_value(value):
    if value <= 15:
        return 0
    elif 15 < value <= 50:
        return 5
    else:  # value > 50
        return 2

# Apply the transformation to all columns labeled from 1 to 12
for col in df.columns:
    if col.isdigit() and int(col) in range(1, 13):  # Check if the column name is a digit and in the range 1-12
        df[col] = df[col].apply(transform_value)

# Save the transformed DataFrame to a new CSV file
df.to_csv(diluted, index=False)

print(f'Transformed data saved to {diluted}')
print(df)

Transformed data saved to /Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test_transformed.csv
  column/row  1  2  3  4  5  6  7  8  9  10  11  12
0          A  2  2  2  2  2  5  2  2  2   5   2   5
1          B  2  2  2  2  2  2  2  2  2   2   2   5
2          C  2  2  2  5  2  2  2  2  5   2   2   2
3          D  2  2  2  2  2  2  2  2  2   2   2   2
4          E  2  5  2  2  2  5  5  2  5   2   2   2
5          F  2  2  2  2  5  2  2  5  2   2   2   2
6          G  2  2  2  2  2  5  2  2  2   2   2   2
7          H  2  2  5  2  5  5  2  5  2   2   5   2


plan: in the new plate we will need to add 5 ul of water meaning that the new concentration formula will be 1:2 for the low concentration (1/2 of q_bit concentration), and for over 50 its going to be 2:7 (c2 = (2*c1)/7) 

In [26]:
initial_concentration = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test.csv'

dilution = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test_transformed.csv'

dil_concentration = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/diluted_conc.csv'

icon = pd.read_csv(initial_concentration)
diluted = pd.read_csv(dilution)
dcon = pd.read_csv(dil_concentration)

def concentration_finder(init_con, amount):
    if amount == 5:
        return init_con/2
    elif amount == 2:
        return init_con/7
    else: # amount is 0 
        return 0

# Create a new DataFrame to store the diluted concentrations
diluted_concentrations = pd.DataFrame()

# Iterate through each column in the diluted DataFrame
for col in diluted.columns:
    if col.isdigit() and int(col) in range(1, 13):  # Check if the column name is a digit and in the range 1-12
        # Apply the concentration finder function for each value in the column
        diluted_concentrations[col] = diluted[col].apply(lambda amount: concentration_finder(icon[col], diluted[col]))

# Save the diluted concentrations to a new CSV file
diluted_concentrations.to_csv(dil_concentration, index=False)

print(f'Diluted concentrations saved to {dil_concentration}')



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [27]:
import pandas as pd

# File paths
initial_concentration = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test.csv'
dilution = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/q_bit_test_transformed.csv'
dil_concentration = '/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/csvs for concentrations/diluted_conc.csv'

# Read the CSV files into DataFrames
icon = pd.read_csv(initial_concentration)
diluted = pd.read_csv(dilution)

# Define the concentration finder function
def concentration_finder(init_con, amount):
    if amount == 5:
        return init_con / 2
    elif amount == 2:
        return init_con / 7
    else:  # amount is 0
        return 0

# Create a new DataFrame to store the diluted concentrations
diluted_concentrations = pd.DataFrame()

# Iterate through each column in the diluted DataFrame
for col in diluted.columns:
    if col.isdigit() and int(col) in range(1, 13):  # Check if the column name is a digit and in the range 1-12
        # Apply the concentration finder function for each value in the column
        diluted_concentrations[col] = diluted[col].combine(icon[col], concentration_finder(icon[col], diluted[col]))

# Save the diluted concentrations to a new CSV file
diluted_concentrations.to_csv(dil_concentration, index=False)

print(f'Diluted concentrations saved to {dil_concentration}')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
await lh.setup()

In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    PCR_Plate_96_Well,
    HTF_L,  #""" Tip Rack with 96 1000ul High Volume Tip with filter """
    LT_L,   #""" Tip Rack with 96 10ul Low Volume Tip with filter """
    STF_L,  #""" Tip Rack with 96 300ul Standard Volume Tip with filter """
)

In [ ]:
lh.summary()

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = LT_L(name='tips_10')
tip_car[1] = STF_L(name='tips_300')
tip_car[2] = HTF_L(name='tips_1000')

lh.deck.assign_child_resource(tip_car, rails=23)

plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = PCR_Plate_96_Well(name='final')
plt_car[1] = PCR_Plate_96_Well(name='source')
plt_car[2] = Cos_96_DW_1mL(name='water')

lh.deck.assign_child_resource(plt_car, rails=17)

lh.summary()

In [ ]:
#/Users/danielkozyrytski/Documents/GitHub/pylabrobot_projects/basic operations tests/q_bit_test.csv